## 3. Data Preparation

This script merges all the currency files and also merge the daily level crypto prices and market prices (dow jones indicator).

Furthermore, custom variables are created such as polarity, Bullish Ratio and Negative Share.

**Output is the daily level summarised data required for modeling exercise**

In [1]:
import pandas as pd
import numpy as np
import string
# !pip install --user -U nltk # if you don't have it
import nltk 
#nltk.download() # if you don't have it
from nltk.corpus import stopwords
import nltk.tokenize 
from nltk.stem import WordNetLemmatizer

#word count
from collections import Counter

#word clouds
import heapq
#from wordcloud import WordCloud, STOPWORDS
from PIL import Image
import collections
import matplotlib.cm as cm
from matplotlib import rcParams
from tqdm import tqdm

#sentiment analysis
from textblob import TextBlob

#text preprocessing
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer 
nltk.download('stopwords')
nltk.download('wordnet')
from scipy import stats

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saggarwal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\saggarwal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Cryptocurrency Prices

Daily level crypto prices are downloaded from internet across different exchanges and average prices are calculated to get final bitcoin prices 

In [2]:

Bitstamp_ETHUSD_d = pd.read_csv('Bitstamp_ETHUSD_d.csv', skiprows=1)

gemini_ETHUSD_day = pd.read_csv('gemini_ETHUSD_day.csv', skiprows=1)
gemini_ETHUSD_day.head()


,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
0,1629777600000,2021-08-24 04:00:00,ETHUSD,3326.28,3359.72,3146.34,3179.40,22907.733466
1,1629691200000,2021-08-23 04:00:00,ETHUSD,3343.53,3380.01,3280.09,3326.28,20502.551719
2,1629604800000,2021-08-22 04:00:00,ETHUSD,3228.24,3361.34,3129.11,3343.53,18344.193056
3,1629518400000,2021-08-21 04:00:00,ETHUSD,3278.28,3311.14,3200.00,3228.24,17990.848364
4,1629432000000,2021-08-20 04:00:00,ETHUSD,3217.82,3301.22,3187.16,3278.28,23260.880280


In [3]:
Bitstamp_ETHUSD_d.rename(columns={'close':'Close_Bitstamp', 'Volume ETH':'Volume_Bitstamp', 'open':'Open_Bitstamp',
                                 'high':'High_Bitstamp','low':'Low_Bitstamp'},inplace=True)
gemini_ETHUSD_day.rename(columns={'Close':'Close_gemini', 'Volume':'Volume_gemini','Open':'Open_gemini',
                                 'High':'High_gemini','Low':'Low_gemini'},inplace=True)

In [5]:
gemini_ETHUSD_day.head()

,Unix Timestamp,Date,Symbol,Open_gemini,High_gemini,Low_gemini,Close_gemini,Volume_gemini
0,1629777600000,2021-08-24 04:00:00,ETHUSD,3326.28,3359.72,3146.34,3179.40,22907.733466
1,1629691200000,2021-08-23 04:00:00,ETHUSD,3343.53,3380.01,3280.09,3326.28,20502.551719
2,1629604800000,2021-08-22 04:00:00,ETHUSD,3228.24,3361.34,3129.11,3343.53,18344.193056
3,1629518400000,2021-08-21 04:00:00,ETHUSD,3278.28,3311.14,3200.00,3228.24,17990.848364
4,1629432000000,2021-08-20 04:00:00,ETHUSD,3217.82,3301.22,3187.16,3278.28,23260.880280


In [6]:

gemini_ETHUSD_day['Date'] = pd.to_datetime(gemini_ETHUSD_day['Date']).dt.date
Bitstamp_ETHUSD_d['Date'] = pd.to_datetime(Bitstamp_ETHUSD_d['date']).dt.date

BTC_prices  = gemini_ETHUSD_day[['Date','Open_gemini','High_gemini','Low_gemini','Close_gemini','Volume_gemini']].merge(Bitstamp_ETHUSD_d[['Date','Open_Bitstamp','High_Bitstamp','Low_Bitstamp','Close_Bitstamp','Volume_Bitstamp']], on='Date', how='left')

BTC_prices.head()

,Date,Open_gemini,High_gemini,Low_gemini,Close_gemini,Volume_gemini,Open_Bitstamp,High_Bitstamp,Low_Bitstamp,Close_Bitstamp,Volume_Bitstamp
0,2021-08-24,3326.28,3359.72,3146.34,3179.40,22907.733466,3326.21,3361.13,3145.17,3175.68,23329.900752
1,2021-08-23,3343.53,3380.01,3280.09,3326.28,20502.551719,3242.33,3381.44,3234.12,3317.25,29497.491812
2,2021-08-22,3228.24,3361.34,3129.11,3343.53,18344.193056,3226.23,3276.43,3130.00,3241.55,12365.330864
3,2021-08-21,3278.28,3311.14,3200.00,3228.24,17990.848364,3283.54,3312.53,3209.15,3226.15,14103.885380
4,2021-08-20,3217.82,3301.22,3187.16,3278.28,23260.880280,3184.76,3305.09,3179.87,3285.64,42648.690274


In [7]:
BTC_prices['Volume_BTC_gemini'] = BTC_prices['Volume_gemini']*BTC_prices['Close_gemini']
#BTC_prices['Volume_BTC_Bittrex'] = BTC_prices['Volume_Bittrex']*BTC_prices['Close_Bittrex']
BTC_prices['Volume_BTC_Bitstamp'] = BTC_prices['Volume_Bitstamp']*BTC_prices['Close_Bitstamp']
#BTC_prices['Volume_BTC_Cexio'] = BTC_prices['Volume_Cexio']*BTC_prices['Close_Cexio']
BTC_prices['Close_prices'] = (BTC_prices['Close_gemini']+BTC_prices['Close_Bitstamp'])/2
BTC_prices['Open_prices'] = (BTC_prices['Open_gemini']+BTC_prices['Open_Bitstamp'])/2
BTC_prices['High_prices'] = (BTC_prices['High_gemini']+BTC_prices['High_Bitstamp'])/2
BTC_prices['Low_prices'] = (BTC_prices['Low_gemini']+BTC_prices['Low_Bitstamp'])/2
BTC_prices['Volume_BTC'] = BTC_prices['Volume_BTC_gemini']+BTC_prices['Volume_BTC_Bitstamp']
BTC_prices.head()

,Date,Open_gemini,High_gemini,Low_gemini,Close_gemini,Volume_gemini,Open_Bitstamp,High_Bitstamp,Low_Bitstamp,Close_Bitstamp,Volume_Bitstamp,Volume_BTC_gemini,Volume_BTC_Bitstamp,Close_prices,Open_prices,High_prices,Low_prices,Volume_BTC
0,2021-08-24,3326.28,3359.72,3146.34,3179.40,22907.733466,3326.21,3361.13,3145.17,3175.68,23329.900752,7.283285e+07,7.408830e+07,3177.540,3326.245,3360.425,3145.755,1.469211e+08
1,2021-08-23,3343.53,3380.01,3280.09,3326.28,20502.551719,3242.33,3381.44,3234.12,3317.25,29497.491812,6.819723e+07,9.785055e+07,3321.765,3292.930,3380.725,3257.105,1.660478e+08
2,2021-08-22,3228.24,3361.34,3129.11,3343.53,18344.193056,3226.23,3276.43,3130.00,3241.55,12365.330864,6.133436e+07,4.008284e+07,3292.540,3227.235,3318.885,3129.555,1.014172e+08
3,2021-08-21,3278.28,3311.14,3200.00,3228.24,17990.848364,3283.54,3312.53,3209.15,3226.15,14103.885380,5.807878e+07,4.550125e+07,3227.195,3280.910,3311.835,3204.575,1.035800e+08
4,2021-08-20,3217.82,3301.22,3187.16,3278.28,23260.880280,3184.76,3305.09,3179.87,3285.64,42648.690274,7.625568e+07,1.401282e+08,3281.960,3201.290,3303.155,3183.515,2.163839e+08


### Merge different datasets

Custom variable creating and summary data at daily level is created

In [8]:
def data_manipulation(df):

    df['length_tweet'] = df['tweet_new2'].apply(len)
    df['Date'] = pd.to_datetime(df['created_date']).dt.date
    summarised = df.groupby(['Date'], as_index=False).agg({'neg':'mean','neu':'mean','pos':'mean','compound':'mean','retweet_count':'mean','like_count':'mean','author id':'count',
                                                                                  "unique_id": pd.Series.nunique, 'length_tweet':'mean','followers_count':'mean'})
    
    data_sum = df.groupby(['Date','sentiment'], as_index=False)['unique_id'].count()
    data_sum1 = data_sum.pivot(index='Date', columns='sentiment', values='unique_id')
    data_sum1['Total_hr_tweets'] = data_sum1['negative']+data_sum1['positive']+data_sum1['neutral']
    data_sum1 = data_sum1.reset_index()
    
    summarised = summarised.merge(data_sum1, on='Date', how='left')
 
    summarised['Bullish_Ratio'] = summarised['positive']/summarised['negative']
    summarised['Negative_share'] = summarised['negative']/summarised['Total_hr_tweets']
    summarised = summarised.rename(columns={'neg':'avg_neg_scr','neu':'avg_neu_scr','pos':'avg_pos_scr','compound':'avg_cmp_scr','retweet_count':'avg_retweet_cnt','like_count':'avg_like_cnt','author id':'tweet_vol',
                                                                                  "unique_id": 'uniq_tweet_vol', 'length_tweet':'avg_tweet_len','followers_count':'avg_followers_cnt'})
    #summarised['created_date']=pd.to_datetime(summarised['D']).dt.tz_localize(None)
    return summarised

final_data_senti1=pd.read_csv('final_data_senti_july1.csv')
summarised_1 = data_manipulation(final_data_senti1)

In [9]:
summarised_1.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,avg_followers_cnt,negative,neutral,positive,Total_hr_tweets,Bullish_Ratio,Negative_share
0,2021-07-01,0.013596,0.881327,0.104758,0.248166,1719.199002,1.815065,66326,26231,122.181151,2210.552513,3151,32491,30684,66326,9.737861,0.047508
1,2021-07-02,0.010652,0.899724,0.089444,0.210551,2959.021410,1.490908,80288,26618,124.551104,1783.641466,3368,42809,34111,80288,10.127969,0.041949
2,2021-07-03,0.010377,0.863704,0.125762,0.294886,1659.365537,1.568880,68554,26353,124.226376,1460.016921,2558,29862,36134,68554,14.125880,0.037314
3,2021-07-04,0.011672,0.852152,0.135896,0.321410,2098.621008,2.240905,68413,26811,124.692427,1621.065996,3110,28147,37156,68413,11.947267,0.045459
4,2021-07-05,0.011492,0.862529,0.125765,0.312211,1109.262943,1.544109,80778,38371,135.891802,1402.129874,3494,30033,47251,80778,13.523469,0.043254


In [10]:
final_data_senti1=pd.read_csv('final_data_senti_july31.csv')
summarised_2 = data_manipulation(final_data_senti1)
summarised_2.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,avg_followers_cnt,negative,neutral,positive,Total_hr_tweets,Bullish_Ratio,Negative_share
0,2021-07-15,0.008097,0.881379,0.110368,0.290591,4906.589705,1.434522,90603,28810,124.259837,1679.857687,3270,37664,49669,90603,15.189297,0.036092
1,2021-07-16,0.009157,0.881526,0.109152,0.267047,2279.033839,1.194681,79227,30396,125.941333,1508.159933,3655,34075,41497,79227,11.353488,0.046133
2,2021-07-17,0.012707,0.877569,0.109343,0.262026,1718.214514,1.426582,71175,25911,126.555799,1313.809371,3240,35191,32744,71175,10.106173,0.045522
3,2021-07-18,0.026696,0.840524,0.132634,0.255509,2009.887598,1.276739,115336,31648,124.816571,936.128451,11940,44575,58821,115336,4.926382,0.103524
4,2021-07-19,0.022996,0.881259,0.095494,0.199932,1858.292160,1.515465,89071,29251,124.416735,1401.033906,8106,42683,38282,89071,4.722675,0.091006


In [11]:
final_data_senti1=pd.read_csv('final_data_senti_aug15.csv')
summarised_3 = data_manipulation(final_data_senti1)
summarised_3.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,avg_followers_cnt,negative,neutral,positive,Total_hr_tweets,Bullish_Ratio,Negative_share
0,2021-08-01,0.024675,0.869119,0.105964,0.224850,2026.784679,2.012475,107653,38439,123.741484,1645.139151,9070,42788,55795,107653,6.151599,0.084252
1,2021-08-02,0.058024,0.854418,0.087410,0.072769,1611.562113,1.553074,138184,35985,123.905025,1598.257917,30883,54399,52902,138184,1.712981,0.223492
2,2021-08-03,0.025833,0.906091,0.067998,0.113357,2973.767083,0.910059,202510,43413,123.504701,1196.424868,22415,101821,78274,202510,3.492037,0.110686
3,2021-08-04,0.045255,0.869103,0.085293,0.095703,974.257682,1.630038,154543,48807,127.654368,1551.313026,31135,60874,62534,154543,2.008479,0.201465
4,2021-08-05,0.030090,0.865636,0.104083,0.186433,1455.965817,2.032253,138871,57154,123.976230,2264.649459,18546,45142,75183,138871,4.053866,0.133548


In [12]:
summarised_all=summarised_1.append(summarised_2)
summarised_all=summarised_all.append(summarised_3)
summarised_all.shape

(46, 17)

In [13]:
#summarised_all['created_date']=pd.to_datetime(summarised_all['created_date'])
summarised_all.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,avg_followers_cnt,negative,neutral,positive,Total_hr_tweets,Bullish_Ratio,Negative_share
0,2021-07-01,0.013596,0.881327,0.104758,0.248166,1719.199002,1.815065,66326,26231,122.181151,2210.552513,3151,32491,30684,66326,9.737861,0.047508
1,2021-07-02,0.010652,0.899724,0.089444,0.210551,2959.021410,1.490908,80288,26618,124.551104,1783.641466,3368,42809,34111,80288,10.127969,0.041949
2,2021-07-03,0.010377,0.863704,0.125762,0.294886,1659.365537,1.568880,68554,26353,124.226376,1460.016921,2558,29862,36134,68554,14.125880,0.037314
3,2021-07-04,0.011672,0.852152,0.135896,0.321410,2098.621008,2.240905,68413,26811,124.692427,1621.065996,3110,28147,37156,68413,11.947267,0.045459
4,2021-07-05,0.011492,0.862529,0.125765,0.312211,1109.262943,1.544109,80778,38371,135.891802,1402.129874,3494,30033,47251,80778,13.523469,0.043254


In [14]:
summarised_all = summarised_all.merge(BTC_prices[['Date','Open_prices', 'High_prices','Low_prices','Close_prices','Volume_BTC']], left_on='Date', right_on='Date',how='left')
summarised_all.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,...,neutral,positive,Total_hr_tweets,Bullish_Ratio,Negative_share,Open_prices,High_prices,Low_prices,Close_prices,Volume_BTC
0,2021-07-01,0.013596,0.881327,0.104758,0.248166,1719.199002,1.815065,66326,26231,122.181151,...,32491,30684,66326,9.737861,0.047508,2231.180,2256.155,2050.385,2074.605,1.865522e+08
1,2021-07-02,0.010652,0.899724,0.089444,0.210551,2959.021410,1.490908,80288,26618,124.551104,...,42809,34111,80288,10.127969,0.041949,2074.175,2164.750,2017.015,2143.800,6.643321e+07
2,2021-07-03,0.010377,0.863704,0.125762,0.294886,1659.365537,1.568880,68554,26353,124.226376,...,29862,36134,68554,14.125880,0.037314,2145.035,2240.110,2120.090,2219.615,8.198353e+07
3,2021-07-04,0.011672,0.852152,0.135896,0.321410,2098.621008,2.240905,68413,26811,124.692427,...,28147,37156,68413,11.947267,0.045459,2219.225,2389.170,2200.875,2295.655,9.187930e+07
4,2021-07-05,0.011492,0.862529,0.125765,0.312211,1109.262943,1.544109,80778,38371,135.891802,...,30033,47251,80778,13.523469,0.043254,2294.400,2316.725,2158.565,2218.825,6.685918e+07


In [15]:
summarised_all['Date'] = pd.to_datetime(summarised_all['Date'])
summarised_all

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,...,neutral,positive,Total_hr_tweets,Bullish_Ratio,Negative_share,Open_prices,High_prices,Low_prices,Close_prices,Volume_BTC
0,2021-07-01,0.013596,0.881327,0.104758,0.248166,1719.199002,1.815065,66326,26231,122.181151,...,32491,30684,66326,9.737861,0.047508,2231.180,2256.155,2050.385,2074.605,1.865522e+08
1,2021-07-02,0.010652,0.899724,0.089444,0.210551,2959.021410,1.490908,80288,26618,124.551104,...,42809,34111,80288,10.127969,0.041949,2074.175,2164.750,2017.015,2143.800,6.643321e+07
2,2021-07-03,0.010377,0.863704,0.125762,0.294886,1659.365537,1.568880,68554,26353,124.226376,...,29862,36134,68554,14.125880,0.037314,2145.035,2240.110,2120.090,2219.615,8.198353e+07
3,2021-07-04,0.011672,0.852152,0.135896,0.321410,2098.621008,2.240905,68413,26811,124.692427,...,28147,37156,68413,11.947267,0.045459,2219.225,2389.170,2200.875,2295.655,9.187930e+07
4,2021-07-05,0.011492,0.862529,0.125765,0.312211,1109.262943,1.544109,80778,38371,135.891802,...,30033,47251,80778,13.523469,0.043254,2294.400,2316.725,2158.565,2218.825,6.685918e+07
5,2021-07-06,0.010767,0.864418,0.124667,0.306945,1385.331280,1.475759,85888,33254,126.287130,...,30731,50450,85888,10.718079,0.054804,2217.355,2350.555,2217.300,2331.655,9.264312e+07
6,2021-07-07,0.015956,0.887242,0.096689,0.216140,589.273689,3.167510,69590,33335,130.347938,...,31699,31934,69590,5.360752,0.085601,2330.595,2409.155,2265.235,2284.975,7.125894e+07
7,2021-07-08,0.029575,0.880383,0.089840,0.120384,2406.759747,1.223334,124298,50877,129.034522,...,55420,44235,124298,1.795033,0.198257,2283.640,2290.885,2065.500,2117.700,1.582472e+08
8,2021-07-09,0.024675,0.891219,0.083907,0.127639,1628.213216,1.264657,95021,44278,124.833553,...,47861,32307,95021,2.175116,0.156313,2117.635,2190.560,2057.770,2139.980,9.912796e+07
9,2021-07-10,0.027161,0.852573,0.119906,0.206600,1118.151423,1.010778,91763,48404,138.598836,...,33867,41648,91763,2.563269,0.177065,2140.545,2175.785,2075.000,2100.030,5.662038e+07


In [16]:
summarised_all['polarity'] = stats.gmean(summarised_all.iloc[:, [1,3]], axis=1)

In [17]:
summarised_all.head()
summarised_all.columns

Index(['Date', 'avg_neg_scr', 'avg_neu_scr', 'avg_pos_scr', 'avg_cmp_scr',
       'avg_retweet_cnt', 'avg_like_cnt', 'tweet_vol', 'uniq_tweet_vol',
       'avg_tweet_len', 'avg_followers_cnt', 'negative', 'neutral', 'positive',
       'Total_hr_tweets', 'Bullish_Ratio', 'Negative_share', 'Open_prices',
       'High_prices', 'Low_prices', 'Close_prices', 'Volume_BTC', 'polarity'],
      dtype='object')

In [18]:
summarised_all = summarised_all[summarised_all['Date'] >= '2021-07-01'][['Date', 'avg_neg_scr', 'avg_neu_scr', 'avg_pos_scr', 'avg_cmp_scr',
       'avg_retweet_cnt', 'avg_like_cnt', 'tweet_vol', 'uniq_tweet_vol',
       'avg_tweet_len', 'avg_followers_cnt', 'negative', 'neutral', 'positive',
       'Bullish_Ratio', 'Negative_share', 'Open_prices',
       'High_prices', 'Low_prices', 'Close_prices', 'Volume_BTC', 'polarity']]
summarised_all.to_csv('model_file_final_btc.csv')

In [19]:
summarised_all['pct_change'] = summarised_all.Close_prices.pct_change()

In [20]:
def direction(df):

    if (df['pct_change'] <0):
        return 0
    elif (df['pct_change'] >=0):
        return 1

summarised_all['direction']= summarised_all.apply(direction, axis = 1)

In [21]:
summarised_all.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,...,Bullish_Ratio,Negative_share,Open_prices,High_prices,Low_prices,Close_prices,Volume_BTC,polarity,pct_change,direction
0,2021-07-01,0.013596,0.881327,0.104758,0.248166,1719.199002,1.815065,66326,26231,122.181151,...,9.737861,0.047508,2231.180,2256.155,2050.385,2074.605,1.865522e+08,0.037740,NaN,NaN
1,2021-07-02,0.010652,0.899724,0.089444,0.210551,2959.021410,1.490908,80288,26618,124.551104,...,10.127969,0.041949,2074.175,2164.750,2017.015,2143.800,6.643321e+07,0.030866,0.033353,1.0
2,2021-07-03,0.010377,0.863704,0.125762,0.294886,1659.365537,1.568880,68554,26353,124.226376,...,14.125880,0.037314,2145.035,2240.110,2120.090,2219.615,8.198353e+07,0.036124,0.035365,1.0
3,2021-07-04,0.011672,0.852152,0.135896,0.321410,2098.621008,2.240905,68413,26811,124.692427,...,11.947267,0.045459,2219.225,2389.170,2200.875,2295.655,9.187930e+07,0.039826,0.034258,1.0
4,2021-07-05,0.011492,0.862529,0.125765,0.312211,1109.262943,1.544109,80778,38371,135.891802,...,13.523469,0.043254,2294.400,2316.725,2158.565,2218.825,6.685918e+07,0.038017,-0.033468,0.0


### dow jones prices

Daily level market prices are merged with the summarised tweet dataset

In [22]:
dow=pd.read_csv('DOW_Barchart_Interactive_Chart_Daily_08_23_2021.csv', skiprows=[0,615])
dow.tail()

,Date Time,Open,High,Low,Close,Change,Volume,Open Interest
608,2021-08-17,62.69,62.875,61.14,62.31,-1.01,3974400,NaN
609,2021-08-18,61.91,62.830,61.71,61.88,-0.43,4573900,NaN
610,2021-08-19,61.04,61.390,60.15,60.55,-1.33,3981500,NaN
611,2021-08-20,60.41,61.110,60.32,60.89,0.34,3203600,NaN
612,2021-08-23,61.50,62.120,61.46,62.25,1.36,1150601,NaN


In [23]:
dow['Date']=pd.to_datetime(dow['Date Time'])
dow['index_change']=dow['Close'].pct_change()
dow.head()

,Date Time,Open,High,Low,Close,Change,Volume,Open Interest,Date,index_change
0,2019-03-20,52.75,53.50,49.50,49.80,0.00,2350800,NaN,2019-03-20,NaN
1,2019-03-21,49.99,50.00,48.20,48.98,-0.82,1764600,NaN,2019-03-21,-0.016466
2,2019-03-22,48.80,49.95,48.16,48.60,-0.38,844600,NaN,2019-03-22,-0.007758
3,2019-03-25,48.60,49.40,48.00,49.15,0.55,440800,NaN,2019-03-25,0.011317
4,2019-03-26,49.00,49.75,48.18,48.85,-0.30,504700,NaN,2019-03-26,-0.006104


In [24]:
summarised_all=summarised_all.merge(dow[['Date','Close']], on='Date', how='left')
summarised_all['Close'] = summarised_all['Close'].fillna(method='ffill')
summarised_all.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,...,Negative_share,Open_prices,High_prices,Low_prices,Close_prices,Volume_BTC,polarity,pct_change,direction,Close
0,2021-07-01,0.013596,0.881327,0.104758,0.248166,1719.199002,1.815065,66326,26231,122.181151,...,0.047508,2231.180,2256.155,2050.385,2074.605,1.865522e+08,0.037740,NaN,NaN,63.50
1,2021-07-02,0.010652,0.899724,0.089444,0.210551,2959.021410,1.490908,80288,26618,124.551104,...,0.041949,2074.175,2164.750,2017.015,2143.800,6.643321e+07,0.030866,0.033353,1.0,63.51
2,2021-07-03,0.010377,0.863704,0.125762,0.294886,1659.365537,1.568880,68554,26353,124.226376,...,0.037314,2145.035,2240.110,2120.090,2219.615,8.198353e+07,0.036124,0.035365,1.0,63.51
3,2021-07-04,0.011672,0.852152,0.135896,0.321410,2098.621008,2.240905,68413,26811,124.692427,...,0.045459,2219.225,2389.170,2200.875,2295.655,9.187930e+07,0.039826,0.034258,1.0,63.51
4,2021-07-05,0.011492,0.862529,0.125765,0.312211,1109.262943,1.544109,80778,38371,135.891802,...,0.043254,2294.400,2316.725,2158.565,2218.825,6.685918e+07,0.038017,-0.033468,0.0,63.51


In [25]:
summarised_all.to_csv('input_model_file_new.csv')

In [26]:
summarised_all.shape

(46, 25)